In [1]:
# data input schema
input = {
    "a": 5,
    "b": 7
}

In [2]:
type(input)

dict

In [3]:
from pydantic import BaseModel

class CalcInput(BaseModel):
    a: int
    b: int

In [6]:
model_input_1 = CalcInput(a=5, b=7)

In [7]:
input

{'a': 5, 'b': 7}

In [ ]:
# converting model into dictionary
model_input_1.model_dump()

dict

In [12]:
# creating a model from dictionary
model_from_input = CalcInput(**input)

In [13]:
model_from_input

CalcInput(a=5, b=7)

In [17]:
invalid_input_1 = {
    "a": "100a",
    "b": 5
}

In [18]:
model_from_invalid_input_1 = CalcInput(**invalid_input_1)

ValidationError: 1 validation error for CalcInput
a
  Input should be a valid integer, unable to parse string as an integer [type=int_parsing, input_value='100a', input_type=str]
    For further information visit https://errors.pydantic.dev/2.12/v/int_parsing

In [16]:
model_from_invalid_input_1

CalcInput(a=100, b=5)

In [22]:
## Pydantic Field Validators
from pydantic import Field, EmailStr, HttpUrl
class UserInfo(BaseModel):
    username: str = Field(min_length=6, max_length=20)
    email: EmailStr
    password: str = Field(min_length=6, max_length=20)
    profile_page: HttpUrl

In [24]:
# Valid
raw_admin_info = {
    "username": "administrator",
    "email" : "admin@lt.com",
    "password": "administrator@123",
    "profile_page": "https://directai.blog"

}

admin_info = UserInfo(**raw_admin_info)
admin_info

UserInfo(username='administrator', email='admin@lt.com', password='administrator@123', profile_page=HttpUrl('https://directai.blog/'))

In [25]:
# invalid
raw_admin_info = {
    "username": "administrator",
    "email" : "adminlt.com",
    "password": "administrator@123",
    "profile_page": "https://directai.blog"

}

admin_info = UserInfo(**raw_admin_info)
admin_info

ValidationError: 1 validation error for UserInfo
email
  value is not a valid email address: An email address must have an @-sign. [type=value_error, input_value='adminlt.com', input_type=str]

In [26]:
# invalid
raw_admin_info = {
    "username": "administrator",
    "email" : "admin@lt.com",
    "password": "administrator@123",
    "profile_page": "directai.blog"

}

admin_info = UserInfo(**raw_admin_info)
admin_info

ValidationError: 1 validation error for UserInfo
profile_page
  Input should be a valid URL, relative URL without a base [type=url_parsing, input_value='directai.blog', input_type=str]
    For further information visit https://errors.pydantic.dev/2.12/v/url_parsing

In [34]:
from pydantic import BaseModel, Field
from typing import Annotated, Literal

class Item(BaseModel):
    item_id: Annotated[int, Field(gt=0)]
    name: Annotated[str, Field(description="Item name")]
    qty: Annotated[int, Field(ge=1)]

class Order(BaseModel):
    order_id: Annotated[int, Field(gt=0)]
    status: Annotated[
        Literal["delivered", "processing", "cancelled", "not_delivered"], 
        Field(description="Allowed status")]
    items: Annotated[list[Item], Field(description="Items ordered")]
    total: Annotated[float, Field(ge=1, description="Total bill")]
    

In [35]:
raw_order = {
      "order_id": 1001,
      "status": "processing",
      "items": [
        {
          "item_id": 1,
          "name": "Laptop",
          "qty": 1
        },
        {
          "item_id": 2,
          "name": "Mouse",
          "qty": 2
        }
      ],
      "total": 1200
    }

processed_order = Order(**raw_order)

In [29]:
processed_order

Order(order_id=1001, status='processing', items=[Item(item_id=1, name='Laptop', qty=1), Item(item_id=2, name='Mouse', qty=2)])

In [30]:
processed_order.items

[Item(item_id=1, name='Laptop', qty=1), Item(item_id=2, name='Mouse', qty=2)]

In [36]:
raw_order = {
      "order_id": "a1001",
      "status": "processing",
      "items": [
        {
          "item_id": 1,
          "name": "Laptop",
          "qty": 1
        },
        {
          "item_id": 2,
          "name": "Mouse",
          "qty": 2
        }
      ],
      "total": 1200
    }

processed_order = Order(**raw_order)

ValidationError: 1 validation error for Order
order_id
  Input should be a valid integer, unable to parse string as an integer [type=int_parsing, input_value='a1001', input_type=str]
    For further information visit https://errors.pydantic.dev/2.12/v/int_parsing